In [22]:
import pandas as pd
import numpy as np

basics_url="https://datasets.imdbws.com/title.basics.tsv.gz"
akas_url="https://datasets.imdbws.com/title.akas.tsv.gz"
ratings_url="https://datasets.imdbws.com/title.ratings.tsv.gz"



## Converting urls into databases & loading the data

In [23]:
basics = pd.read_csv(basics_url,sep='\t',low_memory=False)
akas = pd.read_csv(akas_url,sep='\t',low_memory=False)
ratings = pd.read_csv(ratings_url,sep='\t',low_memory=False)

## Preprocessing

### Title Basics

#### Replacing "\N" with np.nan

In [24]:
basics = basics.replace({'\\N':np.nan})

#### Eliminating movies that are null for runtimeMinutes

In [25]:
# Checking to see how many null values there are in runtimeMinutes column
basics['runtimeMinutes'].isnull().value_counts()

True     6937611
False    2900117
Name: runtimeMinutes, dtype: int64

In [26]:
# Dropping null values in runtimeMinutes column
basics = basics.dropna(subset='runtimeMinutes')

In [27]:
# Double checking null values are deleted
basics['runtimeMinutes'].isnull().value_counts()


False    2900117
Name: runtimeMinutes, dtype: int64

#### Eliminating movies that are null for genre

In [28]:
# Checking null values in genre column
basics['genres'].isnull().value_counts()

False    2823465
True       76652
Name: genres, dtype: int64

In [29]:
# Dropping null values in genres column
basics = basics.dropna(subset='genres')

In [30]:
# Double checking null values in genre column are deleted
basics['genres'].isnull().value_counts()

False    2823465
Name: genres, dtype: int64

####  Keeping only titleType==Movie

In [31]:
basics['titleType'].value_counts()

tvEpisode       1434094
short            600617
movie            382047
video            180365
tvMovie           91529
tvSeries          90383
tvSpecial         18128
tvMiniSeries      17170
tvShort            8809
videoGame           323
Name: titleType, dtype: int64

In [32]:
titleType_movie = basics.loc[basics['titleType']=='movie']

#### Keeping only movies with startYear from 2000-2022

In [33]:
# checking data types
titleType_movie.dtypes

tconst            object
titleType         object
primaryTitle      object
originalTitle     object
isAdult           object
startYear         object
endYear           object
runtimeMinutes    object
genres            object
dtype: object

In [34]:
# checking how many null values in startYear Column
titleType_movie.isnull().sum()

tconst                 0
titleType              0
primaryTitle           0
originalTitle          0
isAdult                0
startYear           6441
endYear           382047
runtimeMinutes         0
genres                 0
dtype: int64

In [35]:
# dropping null values in startYear Column
titleType_movie = titleType_movie.dropna(subset='startYear')

In [36]:
titleType_movie.isnull().sum()

tconst                 0
titleType              0
primaryTitle           0
originalTitle          0
isAdult                0
startYear              0
endYear           375606
runtimeMinutes         0
genres                 0
dtype: int64

In [37]:
# converting startYear column to int data type
titleType_movie['startYear'] = titleType_movie['startYear'].astype(float)

In [38]:
# double checking startYear column conversion
titleType_movie.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 375606 entries, 8 to 9837678
Data columns (total 9 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   tconst          375606 non-null  object 
 1   titleType       375606 non-null  object 
 2   primaryTitle    375606 non-null  object 
 3   originalTitle   375606 non-null  object 
 4   isAdult         375606 non-null  object 
 5   startYear       375606 non-null  float64
 6   endYear         0 non-null       object 
 7   runtimeMinutes  375606 non-null  object 
 8   genres          375606 non-null  object 
dtypes: float64(1), object(8)
memory usage: 28.7+ MB


In [39]:
# keeping movies from startYear 2000-2022
basics = titleType_movie.loc[(titleType_movie['startYear'] >= 2000) & (titleType_movie['startYear'] <= 2022) ]

#### Eliminating movies that include "Documentary" in genre

In [40]:
# Excluding movies that are included in the documentary category.
is_documentary = basics['genres'].str.contains('documentary',case=False)
basics = basics[~is_documentary]

#### Keeping only US movies

In [41]:
# Filtering the basics table down to only include the US movies by using the filter akas dataframe
keepers =basics['tconst'].isin(akas['titleId'])
keepers



34803      True
61116      True
67669      True
86801      True
93938      True
           ... 
9837401    True
9837410    True
9837449    True
9837494    True
9837578    True
Name: tconst, Length: 147463, dtype: bool

In [42]:
# Filtering the basics database
basics = basics[keepers]

### Title AKAs

#### Replacing "\N" with np.nan

In [43]:
akas = akas.replace({'\\N':np.nan})

#### Keeping only US movies

In [44]:
akas_US = akas.loc[akas['region'] =="US"]

In [45]:
akas = akas_US

In [46]:
akas

,titleId,ordering,title,region,language,types,attributes,isOriginalTitle
5,tt0000001,6,Carmencita,US,NaN,imdbDisplay,NaN,0
14,tt0000002,7,The Clown and His Dogs,US,NaN,NaN,literal English title,0
33,tt0000005,10,Blacksmith Scene,US,NaN,imdbDisplay,NaN,0
36,tt0000005,1,Blacksmithing Scene,US,NaN,alternative,NaN,0
41,tt0000005,6,Blacksmith Scene #1,US,NaN,alternative,NaN,0
...,...,...,...,...,...,...,...,...
35852632,tt9916560,1,March of Dimes Presents: Once Upon a Dime,US,NaN,imdbDisplay,NaN,0
35852702,tt9916620,1,The Copeland Case,US,NaN,imdbDisplay,NaN,0
35852791,tt9916702,1,Loving London: The Playground,US,NaN,NaN,NaN,0
35852834,tt9916756,1,Pretty Pretty Black Girl,US,NaN,imdbDisplay,NaN,0


### Title Ratings

#### Replacing "\N" with np.nan

In [47]:
ratings = ratings.replace({'\\N':np.nan})

#### Keeping only US movies

In [48]:
# Filtering the basics table down to only include the US movies by using the filter akas dataframe
keepers2 =ratings['tconst'].isin(akas['titleId'])


In [49]:
# Filtering the basics database
ratings=ratings[keepers2]

In [50]:
ratings

,tconst,averageRating,numVotes
0,tt0000001,5.7,1969
1,tt0000002,5.8,263
4,tt0000005,6.2,2612
5,tt0000006,5.1,181
6,tt0000007,5.4,818
...,...,...,...
1308704,tt9916200,8.1,229
1308705,tt9916204,8.1,262
1308712,tt9916348,8.3,18
1308713,tt9916362,6.4,5320


## Exporting the csv files

In [52]:
basics.to_csv(r'/Users/faris/Documents/GitHub/IMDB/part 1 csvs/basics.csv', index=False)
akas.to_csv(r'/Users/faris/Documents/GitHub/IMDB/part 1 csvs/akas.csv', index=False)
ratings.to_csv(r'/Users/faris/Documents/GitHub/IMDB/part 1 csvs/ratings.csv', index=False)

In [54]:
## Save current dataframe to file.
basics.to_csv("/Users/faris/Documents/GitHub/IMDB/part 1 csvs/basics.csv.gz",compression='gzip',index=False)

akas.to_csv("/Users/faris/Documents/GitHub/IMDB/part 1 csvs/akas.csv.gz",compression='gzip',index=False)

ratings.to_csv("/Users/faris/Documents/GitHub/IMDB/part 1 csvs/ratings.csv.gz",compression='gzip',index=False)



In [55]:
basics.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
34803,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001.0,NaN,118,"Comedy,Fantasy,Romance"
61116,tt0062336,movie,The Tango of the Widower and Its Distorting Mi...,El tango del viudo y su espejo deformante,0,2020.0,NaN,70,Drama
67669,tt0069049,movie,The Other Side of the Wind,The Other Side of the Wind,0,2018.0,NaN,122,Drama
86801,tt0088751,movie,The Naked Monster,The Naked Monster,0,2005.0,NaN,100,"Comedy,Horror,Sci-Fi"
93938,tt0096056,movie,Crime and Punishment,Crime and Punishment,0,2002.0,NaN,126,Drama
